In [2]:
!pip install nba-api
!pip install requests
!pip install seaborn
!pip install -U scikit-learn

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Tuple, Union, Optional

pd.set_option("display.max_columns", None)


In [4]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard


In [5]:
def get_current_season():
    """Get current NBA season in the format 'YYYY-YY'"""
    today = datetime.now()
    year = today.year
    month = today.month
    
    # NBA season typically starts in October
    # If current month is before October, we're in the second half of the season
    if month < 10:
        season_start = year - 1
    else:
        season_start = year
    
    season_end = str(season_start + 1)[-2:]  # Get last 2 digits
    return f"{season_start}-{season_end}"

In [78]:
class Teams():
    def __init__(self):
        self.current_season = get_current_season()

    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_all_teams(self) -> pd.DataFrame:
        try:
            all_teams = teams.get_teams()
            teams_df = pd.DataFrame(all_teams)
            return teams_df.to_json(orient="records")
        except Exception as e:
            print(f"Error retrieving all teams: {e}")
            raise e
    
    def get_team_details_by_abbreviation(self, teamAbbreviation:str) -> Dict:
        try:
            team_details = teams.find_team_by_abbreviation(teamAbbreviation)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for abbreviation {teamAbbreviation}: {e}")
            raise e
    
    def get_team_details_by_id(self, team_id:int) -> Dict:
        try:
            team_details = teams.find_team_name_by_id(team_id)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for ID {team_id}: {e}")
            raise e
        
    def get_team_by_name(self, team_name:str) -> Dict:
        try:
            team_details = teams.find_teams_by_full_name(team_name)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for name {team_name}: {e}")
            raise e

    def get_team_roster_per_season(self, teamAbbreviation:str, season:str = None) -> pd.DataFrame:
        try:
            season = self.check_valid_season(season)

            team_details = teams.find_team_by_abbreviation(teamAbbreviation)
            team_id = team_details["id"]
            roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
            roster_data = roster.get_data_frames()[0]
            drop_cols = ['']
            roster_data = roster_data[["PLAYER", "NUM", "POSITION", "HEIGHT", "WEIGHT", "BIRTH_DATE", "AGE", "EXP", "SCHOOL"]]
            print(season)
            return roster_data
        except Exception as e:
            print(f"Error retrieving roster for team {teamAbbreviation} in season {season}: {e}")
            raise e

    def get_team_games_played_per_season(self, season:str = None, teamAbbreviation:str = None) -> pd.DataFrame:
        season = self.check_valid_season(season)
        team_details = teams.find_team_by_abbreviation(teamAbbreviation)
        team_id = team_details["id"]
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]
        return game_log
        
    def get_todays_games(self) -> None:
        f = "{gameId}: {awayTeam} @ {homeTeam} : {gameTimeLTZ}" 
        board = scoreboard.ScoreBoard()
        print("ScoreBoardDate: " + board.score_board_date)
        games = board.games.get_dict()
        for game in games:
            gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
            print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

    

In [85]:
#Teams().get_all_teams()
#Teams().get_team_details("LAL")
#print(Teams().get_team_roster_per_season("LAL").columns)
#Teams().get_team_roster_per_season("LAL", season=None)
Teams().get_team_games_played_per_season(season="2024", teamAbbreviation="LAL")

#Teams().get_team_by_name("Los Angeles Lakers")
#Teams().get_team_details_by_id(1610612747)
#Teams().get_team_details_by_abbreviation("LAL")


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612747,0022401199,"APR 13, 2025",LAL @ POR,L,50,32,0.610,240,31,80,0.388,9,28,0.321,10,14,0.714,13,29,42,21,9,11,20,21,81
1,1610612747,0022401185,"APR 11, 2025",LAL vs. HOU,W,50,31,0.617,240,53,87,0.609,19,37,0.514,15,25,0.600,4,38,42,28,11,1,12,15,140
2,1610612747,0022401159,"APR 09, 2025",LAL @ DAL,W,49,31,0.613,240,42,90,0.467,12,36,0.333,16,21,0.762,15,24,39,22,14,0,9,16,112
3,1610612747,0022401153,"APR 08, 2025",LAL @ OKC,L,48,31,0.608,240,37,80,0.463,18,40,0.450,28,37,0.757,12,33,45,21,2,6,18,15,120
4,1610612747,0022401135,"APR 06, 2025",LAL @ OKC,W,48,30,0.615,240,46,84,0.548,22,40,0.550,12,17,0.706,8,35,43,25,6,2,15,15,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,1610612747,0022400118,"OCT 30, 2024",LAL @ CLE,L,3,2,0.600,240,38,77,0.494,6,28,0.214,28,33,0.848,5,32,37,21,6,0,20,7,110
78,1610612747,0022400111,"OCT 28, 2024",LAL @ PHX,L,3,1,0.750,240,39,94,0.415,14,32,0.438,13,18,0.722,11,35,46,25,5,6,11,19,105
79,1610612747,0022400096,"OCT 26, 2024",LAL vs. SAC,W,3,0,1.000,240,48,92,0.522,14,34,0.412,21,24,0.875,13,32,45,32,8,3,19,19,131
80,1610612747,0022400085,"OCT 25, 2024",LAL vs. PHX,W,2,0,1.000,240,40,75,0.533,14,27,0.519,29,39,0.744,6,31,37,33,10,5,12,18,123


'2025-26'